In [1]:
import gym
import numpy as np


In [2]:
maxi = [3, 3, 5.1, 5.1, 3.3, 5.1, 1, 1] 
mini = [-1.7, -1.7, -5.1, -5.1, -3.3, -5.1, 0, 0]
my_bin = [5, 5, 5, 5, 5, 5, 2, 2]

In [3]:
maxi = [1.5, 1.5, 5, 5, 3.14, 5, 1, 1] 
mini = [-1.5, -1.5, -5, -5, -3.14, -5, 0, 0]
my_bin = [10, 10, 10, 10, 10, 10, 2, 2]
#my_bin = [5, 5, 5, 5, 5, 5, 2, 2]

In [4]:
env = gym.make("LunarLander-v2")
observation = env.reset(seed=20, return_info=False)

/home/cloedhaillecourt/anaconda3/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/cloedhaillecourt/anaconda3/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [5]:
test = np.array([-1.6,  1.3999956 , -0.3203935 , -0.48554128,  0.00367194,
        0.072574  ,  0.        ,  0.        ])

In [6]:
def get_discrete_state(obs):
    
    obs = [min(elem_maxi, elem_obs) for elem_obs, elem_maxi in zip(obs, maxi)]
    
    obs = [max(elem_mini, elem_obs) for elem_obs, elem_mini in zip(obs, mini)]
    
    res = np.int64(np.floor(0.99*np.array(my_bin)*(obs-np.array(mini))/(np.array(maxi)-np.array(mini))))
    return res   

In [7]:
res =get_discrete_state(test)

In [8]:
q_table = np.random.uniform(low=0, high=1, size=(my_bin + [env.action_space.n]))
q_table.shape

(10, 10, 10, 10, 10, 10, 2, 2, 4)

In [9]:
LEARNING_RATE = 0.15

DISCOUNT = 0.995
EPISODES = 2000
total = 0
total_reward = 0
prior_reward = 0


epsilon = 1 # Taux d'apprentissage

epsilon_decay_value = 0.99995

In [10]:
import time
import math
rewards = []
cpt_step = 0
for episode in range(EPISODES + 1): #go through the episodes
    print(cpt_step)
    cpt_step = 0
    print(episode)
    t0 = time.time() #set the initial time
    discrete_state = get_discrete_state(env.reset()) # get the discrete start for the restarted environment 
    done = False
    episode_reward = 0 # reward starts as 0 for each episode

    if episode % 2000 == 0: 
        print("Episode: " + str(episode))

    while not done: 
        

        if np.random.random() > epsilon:

            action = np.argmax(q_table[discrete_state]) #take cordinated action
        else:

            action = np.random.randint(0, env.action_space.n) # do a random action

        new_state, reward, done, _ = env.step(action) # step action to get new states, reward, and the "done" status.
        cpt_step += 1
        episode_reward += reward # add the reward

        new_discrete_state = get_discrete_state(new_state)

        if episode % 2000 == 0: # render
            env.render()

        if not done: # update q-table
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[np.array(list(discrete_state) + [action])]
            
            #q_table[discrete_state + (action,)]

            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            q_table[np.array(list(discrete_state) + [action])] = new_q

        discrete_state = new_discrete_state

    if epsilon > 0.01: # epsilon modification
       # if episode_reward > prior_reward and episode > 10000: 
        if episode > 10000:
            epsilon *= 1 - math.pow(5, -6);
            #epsilon = math.pow(epsilon_decay_value, episode - 10000)

            if episode % 500 == 0:
                print("Epsilon: " + str(epsilon))

    t1 = time.time() # episode has finished
    episode_total = t1 - t0 # episode total time
    total = total + episode_total

    total_reward += episode_reward # episode total reward
    prior_reward = episode_reward
    rewards.append(episode_reward)
    if episode % 3 == 0: # every 1000 episodes print the average time and the average reward
        mean = total / 3
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = total_reward / 3
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0

env.close()

0
0
Episode: 0


/home/cloedhaillecourt/anaconda3/lib/python3.9/site-packages/gym/core.py:57: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Time Average: 1.8765689531962078
Mean Reward: -6.820933918175267
58
1
73
2


KeyboardInterrupt: 

In [11]:
env.close()

In [13]:
import pandas as pd

/home/cloedhaillecourt/anaconda3/lib/python3.9/site-packages/numexpr/expressions.py:21: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _np_version_forbids_neg_powint = LooseVersion(numpy.__version__) >= LooseVersion('1.12.0b1')


In [18]:
pd.DataFrame({'rewards': rewards, 'discount' : DISCOUNT, 'learning rate' : LEARNING_RATE}).to_csv('rewards.csv')

,rewards,discount,learning rate
0,-20.462802,0.995,0.15
1,-83.543683,0.995,0.15
